In [1]:
import pickle

with open('/home/vit/Projects/flexibility-analysis/datasets/LIGYSIS/LIGYSIS_sites_DEF_TRANS.pkl', 'rb') as file:
    LIGYSIS_human_chains_per_lig_MAY_2024 = pickle.load(file)
import pandas
df = pandas.DataFrame(LIGYSIS_human_chains_per_lig_MAY_2024)
# Display the loaded data (optional)
print(len(df))
print(df)

8244
         ACC SEG  ID                                             up_aas  \
0     P03372   2   0  [339, 341, 342, 343, 344, 345, 346, 347, 349, ...   
1     P03372   2   1  [329, 330, 331, 332, 333, 335, 336, 337, 338, ...   
2     P03372   2   2                          [326, 327, 328, 330, 352]   
3     P00750   3   0  [342, 357, 358, 401, 402, 403, 482, 484, 485, ...   
4     P02100   1   0  [32, 39, 42, 43, 46, 60, 64, 67, 68, 71, 72, 8...   
...      ...  ..  ..                                                ...   
8239  P40818   3   0           [936, 938, 939, 940, 985, 987, 988, 992]   
8240  P35222   1   0  [205, 206, 207, 208, 211, 242, 243, 245, 246, ...   
8241  O95970   2   0                          [334, 336, 381, 382, 383]   
8242  O95970   2   1                               [422, 423, 424, 425]   
8243  Q9NX01   1   0                                   [93, 96, 97, 99]   

      n_aas    SEG_ID   SITE_NAME pdb_id auth_asym_id struct_asym_id  \
0        61  P03372_2 

In [7]:
index = 4

assert len(df.iloc[index]['aas']) == len(df.iloc[index]['up_aas'])
"+".join([str(i) for i in df.iloc[index]['aas']])

'31+38+41+42+45+59+63+66+67+70+71+85+88+91+92+96+98+102+103+106+137+141'

In [8]:
df['protein_id'] = df['pdb_id'] + df['struct_asym_id']

In [2]:
mapping = {'Aba': 'A', 'Ace': 'X', 'Acr': 'X', 'Ala': 'A', 'Aly': 'K', 'Arg': 'R', 'Asn': 'N', 'Asp': 'D', 'Cas': 'C',
           'Ccs': 'C', 'Cme': 'C', 'Csd': 'C', 'Cso': 'C', 'Csx': 'C', 'Cys': 'C', 'Dal': 'A', 'Dbb': 'T', 'Dbu': 'T',
           'Dha': 'S', 'Gln': 'Q', 'Glu': 'E', 'Gly': 'G', 'Glz': 'G', 'His': 'H', 'Hse': 'S', 'Ile': 'I', 'Leu': 'L',
           'Llp': 'K', 'Lys': 'K', 'Men': 'N', 'Met': 'M', 'Mly': 'K', 'Mse': 'M', 'Nh2': 'X', 'Nle': 'L', 'Ocs': 'C',
           'Pca': 'E', 'Phe': 'F', 'Pro': 'P', 'Ptr': 'Y', 'Sep': 'S', 'Ser': 'S', 'Thr': 'T', 'Tih': 'A', 'Tpo': 'T',
           'Trp': 'W', 'Tyr': 'Y', 'Unk': 'X', 'Val': 'V', 'Ycm': 'C', 'Sec': 'U', 'Pyl': 'O', 'Mhs': 'H', 'Snm': 'S',
           'Mis': 'S', 'Seb': 'S', 'Hic' : 'H', 'Fme': 'M', 'Asb': 'D', 'Sah': 'C', 'Smc': 'C', 'Tpq': 'Y', 'Onl': 'X',
           'Tox': 'W', '5x8': 'X', 'Ddz': 'A', 'Nep': 'X', 'T8l': 'T', 'Cgu': 'E', 'Caf': 'C', 'Dsn': 'S', 'Tys': 'Y',
           'Als': 'S', 'Kcx': 'K', 'P1l': 'C', 'Fgp': 'X', 'Scy': 'C', 'Cxm': 'M', 'Css': 'C', 'Led': 'L', 'Cgl': 'C',
           'Mlz': 'K', 'Scs': 'A'}



## OLD: extract sequences

In [ ]:
import biotite.database.rcsb as rcsb
import biotite.structure.io.pdbx as pdbx
import biotite.structure as struc
import biotite.sequence as seq
import numpy as np
import os, sys
sys.path.append('../../utils/')
import constants


def get_sequence(protein_id):
    mmcif_filepath = f'{constants.CIF_FILES_PATH}/{protein_id[:4]}.cif'

    if not os.path.isfile(mmcif_filepath):
        rcsb.fetch(protein_id[:4], 'cif', target_path=constants.CIF_FILES_PATH)

    mmcif_file = pdbx.CIFFile.read(mmcif_filepath) # reads the entity_poly.pdbx_seq_one_letter_code_can entry.

    chain_mask = mmcif_file.block['pdbx_poly_seq_scheme']['asym_id'].as_array() == protein_id[4:]
    sequence = mmcif_file.block['pdbx_poly_seq_scheme']['mon_id'].as_array()[chain_mask]
    sequence = ''.join([mapping[i.capitalize()] for i in sequence])

    return sequence

df.groupby('ACC')
for accession_id, sequence_group in df.groupby('ACC'):
    # find largest contributor:
    largest_pocket = []
    largest_protein_id = None
    for protein_id, protein_group in sequence_group.groupby('protein_id'):
        if len(set(sum(protein_group['aas'].values.tolist(), []))) > len(largest_pocket):
            largest_pocket = set(sum(protein_group['aas'].values.tolist(), []))
            largest_protein_id = protein_id
    
    if os.path.isfile(f'/home/vit/Projects/flexibility-analysis/datasets/LIGYSIS/sequences/{largest_protein_id}.fasta'):
        continue

    print(f'processing {largest_protein_id}')
    
    sequence = get_sequence(largest_protein_id)

    largest_pocket = [int(''.join(filter(str.isdigit, i))) for i in largest_pocket]
    assert max(largest_pocket) <= len(sequence), f'{max(largest_pocket)} <= {len(sequence)}'
    assert min(largest_pocket) > 0

    with open(f'/home/vit/Projects/flexibility-analysis/datasets/LIGYSIS/sequences/{largest_protein_id}.fasta', 'w') as f:
        f.write(f'>{largest_protein_id}\n')
        f.write(str(sequence))
    np.save(f'/home/vit/Projects/flexibility-analysis/datasets/LIGYSIS/indices/{largest_protein_id}.npy', np.array(list(largest_pocket)))


## Take the Uniprot ID, query the Alphafold-DB, retrieve the predicted structure and map the pocket on the AF-predicted structure

In [ ]:
import requests
import re

base_url = "https://www.ebi.ac.uk/pdbe/"

api_base = base_url + "api/"

uniprot_mapping_url = api_base + 'mappings/uniprot/'

def make_request(url, mode, pdb_id):
    """
    This function can make GET and POST requests to
    the PDBe API
    
    :param url: String,
    :param mode: String,
    :param pdb_id: String
    :return: JSON or None
    """
    if mode == "get":
        response = requests.get(url=url+pdb_id)
    elif mode == "post":
        response = requests.post(url, data=pdb_id)

    if response.status_code == 200:
        return response.json()
    else:
        print("[No data retrieved - %s] %s" % (response.status_code, response.text))
    
    return None

pdb_id = '1a52'
uniprot_id = 'P03372'
struct_asym_id = 'A'
mappings_data = make_request(uniprot_mapping_url, "get", pdb_id)
    
# Check if there is data
if not mappings_data:
    print("No data found")

In [14]:
mappings_data[pdb_id]['UniProt'][uniprot_id]['mappings']

[{'entity_id': 1,
  'chain_id': 'A',
  'struct_asym_id': 'A',
  'unp_start': 297,
  'unp_end': 554,
  'start': {'residue_number': 1,
   'author_residue_number': None,
   'author_insertion_code': ''},
  'end': {'residue_number': 258,
   'author_residue_number': None,
   'author_insertion_code': ''}},
 {'entity_id': 1,
  'chain_id': 'B',
  'struct_asym_id': 'B',
  'unp_start': 297,
  'unp_end': 554,
  'start': {'residue_number': 1,
   'author_residue_number': None,
   'author_insertion_code': ''},
  'end': {'residue_number': 258,
   'author_residue_number': None,
   'author_insertion_code': ''}}]

In [ ]:
uniprot_to_binding_site = {}

for uniprot_id, sequence_group in df.groupby('ACC'):
    # Check if there is data
    for i, row in sequence_group.iterrows():            
        binding_site = row['up_aas']
        if uniprot_id not in uniprot_to_binding_site:
            uniprot_to_binding_site[uniprot_id] = set()
        uniprot_to_binding_site[uniprot_id].update(binding_site)


In [28]:
import numpy as np

for uniprot_id, binding_site in uniprot_to_binding_site.items():
    with open(f'/home/vit/Projects/flexibility-analysis/datasets/LIGYSIS/indices/{uniprot_id}.npy', 'wb') as f:
        np.save(f, np.array(list(binding_site)))

In [ ]:
UniProt_accessions = uniprot_to_binding_site.keys()

def call_api_with_accessions(accessions):

    # Your API endpoint and other parameters
    api_endpoint = "https://alphafold.ebi.ac.uk/api/prediction/"

    for accession in accessions:
      url = f"{api_endpoint}{accession}"
    # Make the API request
      try:
        response = requests.get(url, timeout=10)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Process the API response as needed
            protein_info = response.json()
            pdb_url = protein_info[0].get('pdbUrl')
            pdb_data = requests.get(pdb_url).text
            with open(f'/home/vit/Projects/flexibility-analysis/datasets/LIGYSIS/AF-structures/{accession}.pdb', 'w') as f:
                f.write(pdb_data)
        else:
            # Handle unsuccessful API request
            print(f"API request for {accession} failed with status code: {response.status_code}")

      except Exception as e:
        # Handle exceptions, e.g., connection issues
        print(f"Error occurred for {accession}: {str(e)}")

# Example usage
call_api_with_accessions(UniProt_accessions)


In [47]:
import biotite.structure.io.pdb as pdb
import biotite.structure as struc
import os
import sys
sys.path.append('../../utils/')
import biotite_utils


def load_af_structure(pdb_filepath):
    mmcif_file = pdb.PDBFile.read(pdb_filepath)
    # load file to biotite object
    whole_structure = pdb.get_structure(mmcif_file, model=1, include_bonds=True, extra_fields=['b_factor'])
    return whole_structure[struc.filter_amino_acids(whole_structure)]

af_structures_path = '/home/vit/Projects/flexibility-analysis/datasets/LIGYSIS/AF-structures'
output_dir = '/home/vit/Projects/flexibility-analysis/data/features/plDDT'

for pdb_file in os.listdir(af_structures_path):
    af_structure = load_af_structure(f'{af_structures_path}/{pdb_file}')
    af_structure = biotite_utils.get_protein_backbone(af_structure, 'A')
    # Get the sequence of the protein with the plDDT values
    plDDTs = np.zeros(len(af_structure), dtype=float)
    sequence = ''
    for i, residue in enumerate(af_structure):
        residue_id = mapping[residue.res_name.capitalize()]
        plDDT = residue.b_factor

        plDDTs[i] = plDDT
        sequence += residue_id
    # Save the plDDT values to a file
    np.save(f'{output_dir}/plDDT/{pdb_file.split(".")[0]}.npy', plDDTs)
    # Save the sequence to a file
    with open(f'{output_dir}/sequences/{pdb_file.split(".")[0]}.txt', 'w') as f:
        f.write(str(sequence))


In [50]:
for uniprot_id, binding_site in uniprot_to_binding_site.items():
    if min(binding_site) < 1:
        print(binding_site)

 looks like it is 1-based, not 0-based ..

In [ ]:
import os
import numpy as np

with open(f'/home/vit/Projects/flexibility-analysis/data/features/plDDT/train.txt', 'w') as wf:
    for sequence in os.listdir('/home/vit/Projects/flexibility-analysis/data/features/plDDT/sequences'):
        with open(f'/home/vit/Projects/flexibility-analysis/data/features/plDDT/sequences/{sequence}', 'r') as rf:
            seq = rf.read()
        seq = seq.strip()
        
        binding_site = np.load(f'/home/vit/Projects/flexibility-analysis/datasets/LIGYSIS/indices/{sequence.split(".")[0]}.npy')
        residues = []
        for residue in binding_site:
            if residue > len(seq):
                print(f'{sequence} {residue} > {len(seq)}')
                continue
            residues.append(seq[residue-1] + str(residue - 1))
        if len(residues) == 0:
            print(f'{sequence} has no binding site')
            continue
        wf.write(f'{sequence.split(".")[0]};UNK;UNK;{" ".join(residues)};{seq}\n')

    


O43451.txt 2176 > 1857
O43451.txt 2053 > 1857
O43451.txt 2054 > 1857
O43451.txt 2055 > 1857
O43451.txt 2314 > 1857
O43451.txt 2060 > 1857
O43451.txt 2316 > 1857
O43451.txt 2317 > 1857
O43451.txt 2063 > 1857
O43451.txt 2323 > 1857
O43451.txt 2451 > 1857
O43451.txt 2456 > 1857
O43451.txt 2457 > 1857
O43451.txt 2455 > 1857
O43451.txt 2211 > 1857
O43451.txt 2213 > 1857
O43451.txt 2478 > 1857
O43451.txt 2480 > 1857
O43451.txt 2482 > 1857
O43451.txt 2356 > 1857
O43451.txt 2251 > 1857
O43451.txt 2265 > 1857
O43451.txt 2147 > 1857
O43451.txt 2406 > 1857
O43451.txt 2419 > 1857
O43451.txt 2422 > 1857
O43451.txt 2424 > 1857
O43451.txt 2175 > 1857
P15692.txt 288 > 232
P15692.txt 299 > 232
P15692.txt 300 > 232
P15692.txt 301 > 232
P15692.txt 245 > 232
P15692.txt 248 > 232
P15692.txt 251 > 232
P15692.txt 253 > 232
Q9UPX8.txt 1830 > 1470
Q9UPX8.txt 1800 > 1470
Q9UPX8.txt 1832 > 1470
Q9UPX8.txt 1802 > 1470
Q9UPX8.txt 1803 > 1470
Q9UPX8.txt 1835 > 1470


In [6]:
df[df['ACC'] == 'P15692']

,ACC,SEG,ID,up_aas,n_aas,SEG_ID,SITE_NAME,pdb_id,auth_asym_id,struct_asym_id,rep_chain,RoG,aas,centre,SASA,VOL
470,P15692,1,0,"[245, 248, 251, 253, 288, 299, 300, 301]",8,P15692_1,P15692_1_0,1mkk,B,B,1mkk_B,6.94,"[27, 30, 33, 35, 70, 81, 82, 83]","(0.948, -14.413, 1.174)",383.4,128.0
